In [ ]:
import tweepy
import json
from pymongo import MongoClient

####input your credentials here
consumer_key = 'XXX'
consumer_secret = 'XXX'
access_token = 'XXX'
access_token_secret = 'XXX'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

####input your Mongo credentials here
client = MongoClient(username='YYY', password='YYY')
db = client['Streaming']
collection = db['Hashtags']

# Hashtag target list
TARGETS = ['#14F', '#eleccions14F', '#14-F', '14F', 'eleccions14F', '14-F', '#DebatTV3', '#PresidentIlla', 
           '#FemHo', '#AlCostatDeLaGent', '#JuntsperFer', '#JuntsperSer', '#ParaQueGanemosTodos', '#UnaCataluñaMejor', 
           '#ElCanviQueCatalunyaMereix', '#14FComençaElCanvi', '#RecuperemosCataluña', '#SiThoPensesPDeCAT', '#PerGuanyar', 
           '#UnNouCicle', '#FemFront', '#AlliberaElTeuPaís', '#GuanyaIlla', '#LauraPresidenta', '#VerificacióPB', 
           '#InfoEleccions2021', '#RompeTuVoto', '#Pucherazo', '#JoERC', '#JoEnComuPodem', '#PSC']

print("Tracking: " + str(TARGETS))

In [ ]:
# Create the class that will listen the stream
class StreamListener(tweepy.StreamListener):    

    def on_connect(self):
        print("You are now connected to the streaming API.")
 
    def on_error(self, status_code):
        print('An Error has occured: ' + repr(status_code))
        return False
 
    def on_data(self, data):      
        datajson = json.loads(data)
        text = datajson['text']
        fecha = datajson['created_at']      
        print("Tweet capturado a las "  + str(fecha) + " con el texto " + str(text))    
        collection.insert_one(datajson)

# Need to set 'wait_on_rate_limit=True' to handle Twitter API limitations + retries to handle network errors (among others)
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True, wait_on_rate_limit_notify=True, retry_count=10, retry_delay=5)) 
streamer = tweepy.Stream(auth=auth, listener=listener)

In [ ]:
from urllib3.exceptions import ProtocolError
from urllib3.exceptions import ReadTimeoutError

def start_stream(stream, **kwargs):
    try:
        stream.filter(**kwargs)
#    except KeyboardInterrupt:
#        sys.exit()
    except ReadTimeoutError:
        stream.disconnect()
        print("ReadTimeoutError exception")
        start_stream(stream, **kwargs)
    except ProtocolError:
        stream.disconnect()
        print("Protocol Error exception")
        start_stream(stream, **kwargs)
    except Exception as e:
        print(e)
        stream.disconnect()
        print("Fatal exception")
        start_stream(stream, **kwargs)
        
start_stream(streamer, track=TARGETS)